In [114]:
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from scipy import misc
from sklearn.decomposition import PCA
from numpy.random import RandomState
import re
import numpy as np
import cv2

In [171]:
def get_date_taken(path):
    return Image.open(path)._getexif()[36867]

def gray_scale(images):
    imgs = []
    for i in range(len(images)):
        bw = cv2.cvtColor(images[i,:,:,:], cv2.COLOR_RGB2GRAY)
        imgs.append(bw)
    return np.array(imgs)

def read_images(files: list):
    imgs = []
    labels = []
    for i, f in enumerate(files):
        img = cv2.imread(f)
        width, height, _ = img.shape
        lb = folder_name.match(f).groups()[0]
        if width == 1536 and height == 2048:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        imgs.append(img)
        labels.append(lb)
    imgs = np.array(imgs)
    return imgs, labels

def pre_process(files, func = None):
    images, labels = read_images(files)
    if func:
        return func(images), labels
    return images, labels

def normalize(images):
    imgs = images / 255.
    return imgs.reshape(-1, np.product(imgs.shape[1:]))

In [172]:
folder_name = re.compile(r"data/classifier/(.*)/.*\.jpeg")

files = glob("data/classifier/*/*.jpeg", recursive=True)
files
imgs, label = pre_process(files)
imgs = normalize(imgs)
imgs.shape

(64, 9437184)

In [82]:
# # fig, axes = plt.subplots(27, 3, figsize=(27*10, 3*5))

# fig = plt.figure(figsize=(15, 100))
# ax = []
# count = 1
# for i, img in enumerate(imgs):
#     for c in range(3):
#         ax.append(fig.add_subplot(27,3, count))
#         count += 1
#         plt.imshow(img[c,:,:])
# plt.show()

In [173]:
%%time
from sklearn.decomposition import PCA
pca = PCA(0.80)
transformed = pca.fit_transform(imgs)
pca.explained_variance_ ,pca.explained_variance_ratio_, pca.n_components_

(array([142656.73061749,  59540.29690451,  28908.68473543,  24696.12758275,
         20948.32856587,  14235.82352866,  13253.56768037,  11401.22469702,
         10292.72095383,   8909.51630773,   8314.03467991,   7912.88563689,
          7573.23299722,   7078.21361442,   6526.42101815,   6444.80323743,
          6202.94785791,   5861.95254887,   5582.54756138,   5326.15921365,
          5213.73837872,   5049.45630191,   4846.82378975,   4748.85621242,
          4700.07826775,   4533.47760562]),
 array([0.26497   , 0.11058989, 0.05369487, 0.04587048, 0.03890934,
        0.02644156, 0.02461712, 0.02117659, 0.01911766, 0.0165485 ,
        0.01544245, 0.01469736, 0.01406649, 0.01314704, 0.01212215,
        0.01197055, 0.01152133, 0.01088797, 0.010369  , 0.00989279,
        0.00968398, 0.00937884, 0.00900247, 0.00882051, 0.00872991,
        0.00842046]),
 26)

In [168]:
%%time
from sklearn.cluster import MiniBatchKMeans, KMeans

n_clusters = len(np.unique(labels))
km = MiniBatchKMeans(n_clusters=n_clusters, max_iter=100)
km.fit(transformed)
# km.labels_

CPU times: user 27.8 ms, sys: 5.54 ms, total: 33.3 ms
Wall time: 31.2 ms


MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=5, n_init=3, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=0)

In [169]:
list(zip(km.labels_, labels))

[(2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (2, 'astro-var'),
 (1, 'tlux'),
 (1, 'tlux'),
 (1, 'tlux'),
 (1, 'tlux'),
 (1, 'tlux'),
 (1, 'tlux'),
 (1, 'tlux'),
 (4, 'tlux'),
 (0, 'tlux'),
 (1, 'tlux'),
 (1, 'tlux'),
 (1, 'tlux'),
 (1, 'tlux'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (2, 'astro-v'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (3, 'sepia'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd'),
 (4, 'dd')]

In [104]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
weight = enc.fit_transform(labels)

In [97]:
np.unique(labels)

array(['astro-v', 'astro-var', 'dd', 'tlux'], dtype='<U9')